<a href="https://colab.research.google.com/github/ShovalBenjer/Football-Stats-Analysis-Web-Scraping-KPI-Extraction-and-Machine-Learning/blob/main/Football_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install opencv-python opencv-python-headless


In [3]:
from google.colab.patches import cv2_imshow
from google.colab import drive
import cv2
import numpy as np
import os

In [4]:
!gdown --id 1cRN-Q46hjOLikgRYo-isn-R-8KfoktPS -O coco.names
!gdown --id 1_KLvH9xwYTBhjaXQGkSlvCSdnRBb72aT -O yolov3.cfg
!gdown --id 1LY5x_AodQSe_PSgmMivNy0Qv_WTr4loh -O yolov3.weights


/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1cRN-Q46hjOLikgRYo-isn-R-8KfoktPS
To: /content/coco.names
100% 625/625 [00:00<00:00, 1.58MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1_KLvH9xwYTBhjaXQGkSlvCSdnRBb72aT
To: /content/yolov3.cfg
100% 8.34k/8.34k [00:00<00:00, 19.0MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https:/

In [5]:
# Step 1: Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Step 2: Set file paths
# Update these paths to point to the correct locations in your Google Drive
yolo_weights = 'yolov3.weights'
yolo_config = 'yolov3.cfg'
coco_names = 'coco.names'
video_path = '/content/your_video_file.mp4'  # Update this to the correct path if necessary

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set the file path to your video file
# You will need to get the correct path after mounting
video_path = '/content/drive/My Drive/path_to_your_video_file.mp4'

# Download the file directly from the provided link
!gdown --id 1QKGKQvt1skjhSoGZUxAa5WJA0SMNfthv -O /content/your_video_file.mp4

# Update the video path
video_path = '/content/your_video_file.mp4'

print(f"File is available at: {video_path}")

# Now you can proceed with your video processing as planned


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1QKGKQvt1skjhSoGZUxAa5WJA0SMNfthv
From (redirected): https://drive.google.com/uc?id=1QKGKQvt1skjhSoGZUxAa5WJA0SMNfthv&confirm=t&uuid=3a833e1e-ccae-4f05-9e96-6b0c86e658dd
To: /content/your_video_file.mp4
 83% 2.97G/3.57G [00:24<00:04, 137MB/s]

In [ ]:
# Load the YOLO model
net = cv2.dnn.readNet(yolo_weights, yolo_config)
layer_names = net.getLayerNames()

# Since the output is a list of integers, you can access the layer names directly
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load COCO class names
with open(coco_names, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Function to check aspect ratio
def is_standard_aspect_ratio(frame, expected_ratio=16/9, tolerance=0.1):
    height, width, _ = frame.shape
    aspect_ratio = width / height
    return abs(aspect_ratio - expected_ratio) < tolerance

# Function to detect objects and check for the presence of a ball
def detect_objects(frame):
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    detections = net.forward(output_layers)

    objects_detected = []
    height, width, _ = frame.shape
    ball_detected = False

    for detection in detections:
        for object_detected in detection:
            scores = object_detected[5:]
            class_id = int(np.argmax(scores))
            confidence = scores[class_id]
            if confidence > 0.5:  # Confidence threshold
                center_x = int(object_detected[0] * width)
                center_y = int(object_detected[1] * height)
                w = int(object_detected[2] * width)
                h = int(object_detected[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                label = classes[class_id]

                objects_detected.append({
                    "class_id": class_id,
                    "label": label,
                    "confidence": float(confidence),
                    "box": [x, y, w, h]
                })

                # Check if the object is a ball (class ID for ball in COCO dataset is typically 37)
                if label == 'sports ball':
                    ball_detected = True

                # Draw bounding box (optional)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, f"{label}: {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return frame, objects_detected, ball_detected

# Video processing and filtering pipeline
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    saved_frames = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        if not is_standard_aspect_ratio(frame):
            continue

        processed_frame, detections, ball_detected = detect_objects(frame)

        # Skip frames that do not contain a ball
        if not ball_detected:
            continue

        # Save or further process the frame
        saved_frames += 1
        cv2_imshow(processed_frame)  # Updated for Colab
        key = cv2.waitKey(1)
        if key == ord('q'):  # Press 'q' to quit early
            break

        if saved_frames >= 100:  # Process only first 100 relevant frames for PoC
            break

    cap.release()
    cv2.destroyAllWindows()
    print(f"Processed {frame_count} frames, saved {saved_frames} relevant frames.")

# Run the PoC
process_video(video_path)


In [ ]:
print("Unconnected OutLayers:", net.getUnconnectedOutLayers())